# In this notebook, we will be loading all of our previously fitted and saved models and evaluating on the same set of testing data.

In [1]:
# Imports

import os
import pandas as pd
import numpy as np
import keras
import h5py
import pickle

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Random seed for reproducibility
np.random.seed(42)

Using TensorFlow backend.


In [2]:
photos = pd.read_csv('../../../data/yelp/photos_with_labels.csv')
photos['label'] = photos['label'].astype(str)
photos.head()

,photo_id,label
0,--3uF9gf2WttniseRLlI6g.jpg,1
1,--4v2HCSXi092RwfidxlMg.jpg,0
2,--5FyXiUudlvrJyUCvpZJA.jpg,0
3,--6kOAkha3ULjhhw32KN1w.jpg,2
4,--8aLa0rf2gfOjA6kCC3WQ.jpg,1


In [3]:
photos.dtypes

photo_id    object
label       object
dtype: object

In [4]:
main_dir = '../../../data/yelp/photos/'
images = os.listdir(main_dir)
print(len(images))

200000


In [5]:
# Split the data into training and testing sets
# 10,000 testing images

train, test = train_test_split(photos, test_size = 0.05, random_state = 42, stratify = photos['label'])

In [6]:
train.head()

,photo_id,label
11685,2kLHHZhvzRBcCkqrD8BvXQ.jpg,1
13882,3SYgJ2ttrvrkkZQVwBB9ZA.jpg,1
43353,Cv1ONeniReCw_1qi2cljtA.jpg,0
4756,0XjhceFCeCqM08OUm6Oniw.jpg,2
172017,r0cvTXkUdTk5AZ8fgvv6Cw.jpg,0


In [7]:
test.head()

,photo_id,label
44587,DJfdb5ollatiIBcXhb_mVw.jpg,0
37985,BDk3HPd4srLRB34I6m794g.jpg,0
152320,kizRI9rvstjGs_6lyddxNA.jpg,3
4733,0X45rZJBrarNJnOk4sphqA.jpg,0
13497,3KUcJRrR_WYG7eAFZWGz0w.jpg,1


In [8]:
# Instantiate an ImageDataGenerator class to pass all of our images through
# With rescaling

new_datagen=ImageDataGenerator(samplewise_center=True,
                               samplewise_std_normalization=True,
                               rotation_range=20,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               shear_range=16,
                               zoom_range=[0.9, 1.1],
                               fill_mode="constant",
                               cval=255,
                               horizontal_flip=True,
                               vertical_flip=True,
                               rescale=1./255.,
                               validation_split=0.25)

test_datagen = ImageDataGenerator(samplewise_center=True,
                                  samplewise_std_normalization=True,
                                  rotation_range=20,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=16,
                                  zoom_range=[0.9, 1.1],
                                  fill_mode="constant",
                                  cval=255,
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  rescale=1./255.)

In [9]:
# Create Train, Validation, and Testing generators

new_train_generator = new_datagen.flow_from_dataframe(
    dataframe=train,
    directory=main_dir,
    x_col="photo_id",
    y_col="label",
    subset="training",
    batch_size=250,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    interpolation="bilinear",
    target_size=(256, 256)
)

new_valid_generator = new_datagen.flow_from_dataframe(
    dataframe=train,
    directory=main_dir,
    x_col="photo_id",
    y_col="label",
    subset="validation",
    batch_size=250,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    interpolation="bilinear",
    target_size=(256, 256)
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test,
    directory=main_dir,
    x_col="photo_id",
    y_col="label",
    batch_size=1,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    interpolation="bilinear",
    target_size=(256, 256)
)

Found 142500 validated image filenames belonging to 5 classes.
Found 47500 validated image filenames belonging to 5 classes.
Found 10000 validated image filenames belonging to 5 classes.


In [10]:
# Instantiate an ImageDataGenerator class to pass all of our images through
# Without rescaling

nors_datagen=ImageDataGenerator(featurewise_center=True,
                           featurewise_std_normalization=True,
                           rotation_range=20,
                           width_shift_range=0.1,
                           height_shift_range=0.1,
                           shear_range=16,
                           zoom_range=[0.9, 1.1],
                           fill_mode="constant",
                           cval=255,
                           horizontal_flip=True,
                           vertical_flip=True,
                           validation_split=0.25)

nors_test_datagen = ImageDataGenerator(samplewise_center=True,
                                  samplewise_std_normalization=True,
                                  rotation_range=20,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=16,
                                  zoom_range=[0.9, 1.1],
                                  fill_mode="constant",
                                  cval=255,
                                  horizontal_flip=True,
                                  vertical_flip=True)

In [11]:
# Create Train, Validation, and Testing generators

nors_train_generator = nors_datagen.flow_from_dataframe(
                                            dataframe=train,
                                            directory=main_dir,
                                            x_col="photo_id",
                                            y_col="label",
                                            subset="training",
                                            batch_size=100,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="categorical",
                                            interpolation="bilinear",
                                            target_size=(256, 256))
nors_valid_generator = nors_datagen.flow_from_dataframe(
                                            dataframe=train,
                                            directory=main_dir,
                                            x_col="photo_id",
                                            y_col="label",
                                            subset="validation",
                                            batch_size=100,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="categorical",
                                            interpolation="bilinear",
                                            target_size=(256, 256))

nors_test_generator = nors_test_datagen.flow_from_dataframe(
    dataframe=test,
    directory=main_dir,
    x_col="photo_id",
    y_col="label",
    batch_size=1,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    interpolation="bilinear",
    target_size=(256, 256)
)

Found 142500 validated image filenames belonging to 5 classes.
Found 47500 validated image filenames belonging to 5 classes.
Found 10000 validated image filenames belonging to 5 classes.


## We will seek to see if rescaling images has a significant impact in our predictions.

### First Model

In [12]:
# Load the pickled model

with open('../../../first_model.pkl', 'rb') as f:
    model1 = pickle.load(f)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [13]:
# Model summary

model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 125, 125, 64)      9280      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 246016)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                15745088  
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
__________

In [14]:
model1.evaluate_generator(test_generator, steps = len(test_generator), verbose = 1)

10000/10000 [==============================] - 319s 32ms/step


[1.1065300325712801, 0.6867]

In [15]:
# With rescaled images

Y_pred = model1.predict_generator(test_generator, len(test_generator))
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['Food', 'Inside', 'Drink', 'Outside', 'Menu']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Confusion Matrix
[[4401 1337    6    0    0]
 [2046  573    3    0    0]
 [ 698  207    1    0    0]
 [ 447  127    3    0    0]
 [ 114   37    0    0    0]]
Classification Report
              precision    recall  f1-score   support

        Food       0.57      0.77      0.65      5744
      Inside       0.25      0.22      0.23      2622
       Drink       0.08      0.00      0.00       906
     Outside       0.00      0.00      0.00       577
        Menu       0.00      0.00      0.00       151

    accuracy                           0.50     10000
   macro avg       0.18      0.20      0.18     10000
weighted avg       0.40      0.50      0.44     10000



C:\Users\bruno\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [19]:
model1.evaluate_generator(nors_test_generator, steps = len(nors_test_generator), verbose = 1)

10000/10000 [==============================] - 327s 33ms/step


[1.0953757158050121, 0.6937]

In [16]:
# With non-rescaled images

Y_pred = model1.predict_generator(nors_test_generator, len(nors_test_generator))
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(nors_test_generator.classes, y_pred))
print('Classification Report')
target_names = ['Food', 'Inside', 'Drink', 'Outside', 'Menu']
print(classification_report(nors_test_generator.classes, y_pred, target_names=target_names))

Confusion Matrix
[[4445 1292    7    0    0]
 [2036  583    3    0    0]
 [ 719  187    0    0    0]
 [ 443  130    4    0    0]
 [ 110   40    1    0    0]]
Classification Report
              precision    recall  f1-score   support

        Food       0.57      0.77      0.66      5744
      Inside       0.26      0.22      0.24      2622
       Drink       0.00      0.00      0.00       906
     Outside       0.00      0.00      0.00       577
        Menu       0.00      0.00      0.00       151

    accuracy                           0.50     10000
   macro avg       0.17      0.20      0.18     10000
weighted avg       0.40      0.50      0.44     10000



C:\Users\bruno\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Second Model

In [17]:
# Load the pickled model

with open('Models/3conv2densemodel15epoch.pkl', 'rb') as f:
    model2 = pickle.load(f)

In [18]:
# Model summary

model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 125, 125, 32)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 60, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 57600)             0         
__________

In [20]:
model2.evaluate_generator(test_generator, steps = len(test_generator), verbose = 1)

10000/10000 [==============================] - 354s 35ms/step


[0.3949575798887481, 0.8712]

In [21]:
model2.evaluate_generator(nors_test_generator, steps = len(nors_test_generator), verbose = 1)

10000/10000 [==============================] - 421s 42ms/step


[0.39495747243667617, 0.8712]

In [22]:
# With rescaled images

Y_pred = model2.predict_generator(test_generator, len(test_generator))
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['Food', 'Inside', 'Drink', 'Outside', 'Menu']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Confusion Matrix
[[3510 1601  315  265   53]
 [1605  756  131  107   23]
 [ 552  275   45   29    5]
 [ 359  166   27   19    6]
 [  99   37    6    6    3]]
Classification Report
              precision    recall  f1-score   support

        Food       0.57      0.61      0.59      5744
      Inside       0.27      0.29      0.28      2622
       Drink       0.09      0.05      0.06       906
     Outside       0.04      0.03      0.04       577
        Menu       0.03      0.02      0.02       151

    accuracy                           0.43     10000
   macro avg       0.20      0.20      0.20     10000
weighted avg       0.41      0.43      0.42     10000



In [23]:
# With non-rescaled images

Y_pred = model2.predict_generator(nors_test_generator, len(nors_test_generator))
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(nors_test_generator.classes, y_pred))
print('Classification Report')
target_names = ['Food', 'Inside', 'Drink', 'Outside', 'Menu']
print(classification_report(nors_test_generator.classes, y_pred, target_names=target_names))

Confusion Matrix
[[3510 1601  315  265   53]
 [1605  756  131  107   23]
 [ 552  275   45   29    5]
 [ 359  166   27   19    6]
 [  99   37    6    6    3]]
Classification Report
              precision    recall  f1-score   support

        Food       0.57      0.61      0.59      5744
      Inside       0.27      0.29      0.28      2622
       Drink       0.09      0.05      0.06       906
     Outside       0.04      0.03      0.04       577
        Menu       0.03      0.02      0.02       151

    accuracy                           0.43     10000
   macro avg       0.20      0.20      0.20     10000
weighted avg       0.41      0.43      0.42     10000



### Third model

In [24]:
# Load the pickled model

with open('Models/3conv2densemodel_2_30epochs.pkl', 'rb') as f:
    model3 = pickle.load(f)

In [25]:
# Model summary

model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 254, 254, 16)      448       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 127, 127, 16)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 125, 125, 32)      4640      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 60, 60, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 57600)             0         
__________

In [26]:
model3.evaluate_generator(test_generator, steps = len(test_generator), verbose = 1)

10000/10000 [==============================] - 328s 33ms/step


[0.3575285795898364, 0.8733]

In [27]:
model3.evaluate_generator(nors_test_generator, steps = len(nors_test_generator), verbose = 1)

10000/10000 [==============================] - 326s 33ms/step


[0.35752856176394654, 0.8733]

In [28]:
# With rescaled images

Y_pred = model3.predict_generator(test_generator, len(test_generator))
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['Food', 'Inside', 'Drink', 'Outside', 'Menu']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Confusion Matrix
[[3564 1521  376  200   83]
 [1613  695  177  104   33]
 [ 575  231   65   24   11]
 [ 357  148   40   24    8]
 [  95   38   10    5    3]]
Classification Report
              precision    recall  f1-score   support

        Food       0.57      0.62      0.60      5744
      Inside       0.26      0.27      0.26      2622
       Drink       0.10      0.07      0.08       906
     Outside       0.07      0.04      0.05       577
        Menu       0.02      0.02      0.02       151

    accuracy                           0.44     10000
   macro avg       0.20      0.20      0.20     10000
weighted avg       0.41      0.44      0.42     10000



In [29]:
# With non-rescaled images

Y_pred = model3.predict_generator(nors_test_generator, len(nors_test_generator))
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(nors_test_generator.classes, y_pred))
print('Classification Report')
target_names = ['Food', 'Inside', 'Drink', 'Outside', 'Menu']
print(classification_report(nors_test_generator.classes, y_pred, target_names=target_names))

Confusion Matrix
[[3564 1521  376  200   83]
 [1613  695  177  104   33]
 [ 575  231   65   24   11]
 [ 357  148   40   24    8]
 [  95   38   10    5    3]]
Classification Report
              precision    recall  f1-score   support

        Food       0.57      0.62      0.60      5744
      Inside       0.26      0.27      0.26      2622
       Drink       0.10      0.07      0.08       906
     Outside       0.07      0.04      0.05       577
        Menu       0.02      0.02      0.02       151

    accuracy                           0.44     10000
   macro avg       0.20      0.20      0.20     10000
weighted avg       0.41      0.44      0.42     10000



### Fourth model

In [31]:
# Load the h5py instance of the model using keras.models.load_model

model4 = keras.models.load_model('Models/new_model_for_testing.h5')

In [32]:
# Look at the model summary

model4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 254, 254, 16)      448       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 127, 127, 16)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 125, 125, 32)      4640      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 60, 60, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 57600)             0         
__________

In [33]:
model4.evaluate_generator(test_generator, steps = len(test_generator), verbose = 1)

10000/10000 [==============================] - 329s 33ms/step


[0.35494111035507797, 0.8754]

In [34]:
model4.evaluate_generator(nors_test_generator, steps = len(nors_test_generator), verbose = 1)

10000/10000 [==============================] - 336s 34ms/step


[0.3549411430792122, 0.8754]

In [35]:
# With rescaled images

Y_pred = model4.predict_generator(test_generator, len(test_generator))
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['Food', 'Inside', 'Drink', 'Outside', 'Menu']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Confusion Matrix
[[3663 1564  239  197   81]
 [1655  765   95   73   34]
 [ 579  236   48   37    6]
 [ 361  159   29   21    7]
 [ 100   39    5    5    2]]
Classification Report
              precision    recall  f1-score   support

        Food       0.58      0.64      0.61      5744
      Inside       0.28      0.29      0.28      2622
       Drink       0.12      0.05      0.07       906
     Outside       0.06      0.04      0.05       577
        Menu       0.02      0.01      0.01       151

    accuracy                           0.45     10000
   macro avg       0.21      0.21      0.20     10000
weighted avg       0.42      0.45      0.43     10000



In [36]:
# With non-rescaled images

Y_pred = model4.predict_generator(nors_test_generator, len(nors_test_generator))
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(nors_test_generator.classes, y_pred))
print('Classification Report')
target_names = ['Food', 'Inside', 'Drink', 'Outside', 'Menu']
print(classification_report(nors_test_generator.classes, y_pred, target_names=target_names))

Confusion Matrix
[[3663 1564  239  197   81]
 [1655  765   95   73   34]
 [ 579  236   48   37    6]
 [ 361  159   29   21    7]
 [ 100   39    5    5    2]]
Classification Report
              precision    recall  f1-score   support

        Food       0.58      0.64      0.61      5744
      Inside       0.28      0.29      0.28      2622
       Drink       0.12      0.05      0.07       906
     Outside       0.06      0.04      0.05       577
        Menu       0.02      0.01      0.01       151

    accuracy                           0.45     10000
   macro avg       0.21      0.21      0.20     10000
weighted avg       0.42      0.45      0.43     10000



### Fifth Model

In [37]:
# Load the h5py instance of the model using keras.models.load_model

model5 = keras.models.load_model('Models/big_conv_model.h5')

In [38]:
# Model Summary

model5.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 254, 254, 16)      448       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 127, 127, 16)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 125, 125, 32)      4640      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 123, 123, 32)      9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 61, 61, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 59, 59, 64)        18496     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 57, 57, 64)        36928     
__________

In [39]:
# With rescaled images

Y_pred = model5.predict_generator(test_generator, len(test_generator))
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['Food', 'Inside', 'Drink', 'Outside', 'Menu']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Confusion Matrix
[[3471 1558  317  300   98]
 [1599  716  152  134   21]
 [ 557  239   58   40   12]
 [ 345  158   38   24   12]
 [  93   34   11   10    3]]
Classification Report
              precision    recall  f1-score   support

        Food       0.57      0.60      0.59      5744
      Inside       0.26      0.27      0.27      2622
       Drink       0.10      0.06      0.08       906
     Outside       0.05      0.04      0.04       577
        Menu       0.02      0.02      0.02       151

    accuracy                           0.43     10000
   macro avg       0.20      0.20      0.20     10000
weighted avg       0.41      0.43      0.42     10000



In [40]:
# With non-rescaled images

Y_pred = model5.predict_generator(nors_test_generator, len(nors_test_generator))
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(nors_test_generator.classes, y_pred))
print('Classification Report')
target_names = ['Food', 'Inside', 'Drink', 'Outside', 'Menu']
print(classification_report(nors_test_generator.classes, y_pred, target_names=target_names))

Confusion Matrix
[[3471 1558  317  300   98]
 [1599  716  152  134   21]
 [ 557  239   58   40   12]
 [ 345  158   38   24   12]
 [  93   34   11   10    3]]
Classification Report
              precision    recall  f1-score   support

        Food       0.57      0.60      0.59      5744
      Inside       0.26      0.27      0.27      2622
       Drink       0.10      0.06      0.08       906
     Outside       0.05      0.04      0.04       577
        Menu       0.02      0.02      0.02       151

    accuracy                           0.43     10000
   macro avg       0.20      0.20      0.20     10000
weighted avg       0.41      0.43      0.42     10000



### Sixth Model - Balanced

In [41]:
# Load the h5py instance of the model using keras.models.load_model

bal_mod = keras.models.load_model('Models/balanced_model.h5')

In [42]:
# Model Summary

bal_mod.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 254, 254, 16)      448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 127, 127, 16)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 125, 125, 32)      4640      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 123, 123, 32)      9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 61, 61, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 59, 59, 64)        18496     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 57, 57, 64)        36928     
__________

In [43]:
# With rescaled images

Y_pred = bal_mod.predict_generator(test_generator, len(test_generator))
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['Food', 'Inside', 'Drink', 'Outside', 'Menu']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Confusion Matrix
[[   0    0 5573  171    0]
 [   0    0 2548   74    0]
 [   0    0  878   28    0]
 [   0    0  559   18    0]
 [   0    0  143    8    0]]
Classification Report
              precision    recall  f1-score   support

        Food       0.00      0.00      0.00      5744
      Inside       0.00      0.00      0.00      2622
       Drink       0.09      0.97      0.17       906
     Outside       0.06      0.03      0.04       577
        Menu       0.00      0.00      0.00       151

    accuracy                           0.09     10000
   macro avg       0.03      0.20      0.04     10000
weighted avg       0.01      0.09      0.02     10000



C:\Users\bruno\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [44]:
# With non-rescaled images

Y_pred = bal_mod.predict_generator(nors_test_generator, len(nors_test_generator))
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(nors_test_generator.classes, y_pred))
print('Classification Report')
target_names = ['Food', 'Inside', 'Drink', 'Outside', 'Menu']
print(classification_report(nors_test_generator.classes, y_pred, target_names=target_names))

Confusion Matrix
[[   0    0 5573  171    0]
 [   0    0 2548   74    0]
 [   0    0  878   28    0]
 [   0    0  559   18    0]
 [   0    0  143    8    0]]
Classification Report
              precision    recall  f1-score   support

        Food       0.00      0.00      0.00      5744
      Inside       0.00      0.00      0.00      2622
       Drink       0.09      0.97      0.17       906
     Outside       0.06      0.03      0.04       577
        Menu       0.00      0.00      0.00       151

    accuracy                           0.09     10000
   macro avg       0.03      0.20      0.04     10000
weighted avg       0.01      0.09      0.02     10000

